In [1]:
%matplotlib inline
import numpy as np
import pandas as pd 

import os
import matplotlib.pyplot as plt
import PIL
from sklearn.preprocessing import LabelBinarizer

In [2]:
from keras.layers import  Conv3D, MaxPooling3D, Dropout, Dense, Input, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras import initializers
from keras.regularizers import l2
from keras.models import Model
from keras.layers import Reshape, merge, Lambda
from keras.layers.merge import Average
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD

Using TensorFlow backend.


In [3]:
def load_patient(name):
    imglist=[]
    for f in os.listdir('./dataset/'+name+'/')[:8]:
        img = PIL.Image.open('./dataset/'+name+'/{}'.format(f))
        img = img.resize((8,8),PIL.Image.ANTIALIAS)
        img = np.matrix(img)
        imglist.append(img)
    imglist = np.array(imglist)
    return imglist

In [4]:
patient = os.listdir('./dataset/')
patientList=[]
for name in patient:
    imglist = load_patient(name)
    patientList.append(imglist)

In [5]:
patientArray = np.array(patientList)
patientArray = patientArray.reshape(100,8,8,8,1)

In [7]:
lables = pd.read_csv('./lables.csv')
groups = lables['Groups']

lb = LabelBinarizer()
y = lb.fit_transform(groups)

In [8]:
#LightNet

cfg = {'batch_size' : 32, #52, 47892 samples, 921batches
       # 'learning_rate' : lr_schedule,
       'momentum' : 0.9,
       'dims' : (8, 8, 8),
       'n_channels' : 1,
       'n_classes' : 5,
       'batches_per_chunk': 1,#64 -default
       'max_epochs' : 400,# 80 -default
       'max_jitter_ij' : 2,
       'max_jitter_k' : 2,
       'n_rotations' : 12,
       'checkpoint_every_nth' : 4000,
       }




def get_LightNet():
    dims, n_channels, n_classes = tuple(cfg['dims']), cfg['n_channels'], cfg['n_classes']
    n_rotations = cfg['n_rotations']
    #tf(samples,rows,clos,height,channels)
    shape = dims + (n_channels,)

    l_in = Input(shape =shape)
    l_conv1 = Conv3D(
                            filters = 32,
                            kernel_size=(2, 2, 2),
                            padding = 'valid',
                            kernel_initializer = 'he_normal',
                            kernel_regularizer=l2(0.001),
                            name = 'conv1'
                            )(l_in)

    l_conv1 = BatchNormalization()(l_conv1)

    l_out1 = LeakyReLU(alpha = 0.1)(l_conv1)

    l_drop1 = Dropout(rate= 0.1, name = 'drop1')(l_out1)

    l_conv2 = Conv3D(
                            filters=32,# change to 64
                            kernel_size=(2, 2, 2),
                            padding='valid',
                            kernel_initializer='he_normal',
                            kernel_regularizer=l2(0.001),
                            name='conv2'
                            )(l_drop1)

    l_conv2 = BatchNormalization()(l_conv2)

    l_out2 = LeakyReLU(alpha = 0.1)(l_conv2)

    l_pool2 = MaxPooling3D(pool_size = (2, 2, 2), name = 'pool2')(l_out2)

    l_drop2 = Dropout(rate = 0.1, name = 'drop2')(l_pool2)

    #add extra layer compared to original VoxNet

    num_of_subvolumes = 64
    l_conv3 = Conv3D(
        filters=num_of_subvolumes,#we put more kernels here
        kernel_size=(2,2,2),
        padding='valid',
        kernel_initializer='he_normal',
        kernel_regularizer=l2(0.001),
        name='conv3'
    )(l_drop2)

    l_conv3 = BatchNormalization()(l_conv3)

    l_out3 = LeakyReLU(alpha=0.1)(l_conv3)

   # l_pool3 = MaxPooling3D(pool_size=(2, 2, 2), name='pool3')(l_out3)

    l_drop3 = Dropout(rate=0.1, name='drop3')(l_out3)
    #end

    #main branch
    l_flatten = Flatten()(l_drop3)

    l_fc1 = Dense(units = num_of_subvolumes, kernel_initializer = 'he_normal', activation = 'relu', kernel_regularizer=l2(0.001), name = 'fc1')(l_flatten)

    l_drop4 = Dropout(rate = 0.1, name = 'drop4' )(l_fc1)

    main_classification_output = Dense(units = n_classes, kernel_initializer = 'he_normal', activation = 'softmax', kernel_regularizer=l2(0.001), name = 'main_output_40')(l_drop4)
    # main_orientation_output = Dense(units = n_rotations, kernel_initializer = 'he_normal', activation = 'softmax', kernel_regularizer=l2(0.001), name = 'ORN_main_output')(l_drop4)


    #auxiliary supervision

    slicing = Reshape(target_shape = (8,num_of_subvolumes))(l_drop3)

    part0 = Lambda(lambda x: x[:, 0, :], output_shape=(num_of_subvolumes,))(slicing)
    l_subdrop0 = Dropout(rate=0.2, name='subdrop0')(part0)
    sub_output0 = Dense(units=n_classes, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_sub_output0')(l_subdrop0)
    # ORN_output0 = Dense(units=n_rotations, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_ORN_output0')(l_subdrop0)

    part1 = Lambda(lambda x: x[:, 1, :], output_shape=(num_of_subvolumes,))(slicing)
    l_subdrop1 = Dropout(rate=0.1, name='subdrop1')(part1)
    sub_output1 = Dense(units=n_classes, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_sub_output1')(l_subdrop1)
    # ORN_output1 = Dense(units=n_rotations, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_ORN_output1')(l_subdrop1)

    part2 = Lambda(lambda x: x[:, 2, :], output_shape=(num_of_subvolumes,))(slicing)
    l_subdrop2 = Dropout(rate=0.1, name='subdrop2')(part2)
    sub_output2 = Dense(units=n_classes, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_sub_output2')(l_subdrop2)
    # ORN_output2 = Dense(units=n_rotations, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_ORN_output2')(l_subdrop2)

    part3 = Lambda(lambda x: x[:, 3, :], output_shape=(num_of_subvolumes,))(slicing)
    l_subdrop3 = Dropout(rate=0.1, name='subdrop3')(part3)
    sub_output3 = Dense(units=n_classes, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_sub_output3')(l_subdrop3)
    # ORN_output3 = Dense(units=n_rotations, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_ORN_output3')(l_subdrop3)

    part4 = Lambda(lambda x: x[:, 4, :], output_shape=(num_of_subvolumes,))(slicing)
    l_subdrop4 = Dropout(rate=0.1, name='subdrop4')(part4)
    sub_output4 = Dense(units=n_classes, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_sub_output4')(l_subdrop4)
    # ORN_output4 = Dense(units=n_rotations, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_ORN_output4')(l_subdrop4)


    part5 = Lambda(lambda x: x[:, 5, :], output_shape=(num_of_subvolumes,))(slicing)
    l_subdrop5 = Dropout(rate=0.2, name='subdrop5')(part5)
    sub_output5 = Dense(units=n_classes, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_sub_output5')(l_subdrop5)
    # ORN_output5 = Dense(units=n_rotations, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_ORN_output5')(l_subdrop5)


    part6 = Lambda(lambda x: x[:, 6, :], output_shape=(num_of_subvolumes,))(slicing)
    l_subdrop6 = Dropout(rate=0.1, name='subdrop6')(part6)
    sub_output6 = Dense(units=n_classes, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_sub_output6')(l_subdrop6)
    # ORN_output6 = Dense(units=n_rotations, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_ORN_output6')(l_subdrop6)

    part7 = Lambda(lambda x: x[:, 7, :], output_shape=(num_of_subvolumes,))(slicing)
    l_subdrop7 = Dropout(rate=0.1, name='subdrop7')(part7)
    sub_output7 = Dense(units=n_classes, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_sub_output7')(l_subdrop7)
    # ORN_output7 = Dense(units=n_rotations, kernel_initializer='he_normal', activation='softmax', kernel_regularizer=l2(0.001),name = 'ft_ORN_output7')(l_subdrop7)


    #with auxiliary average, we actually give each sub_main_output a weight of 1/8 /2 = 1/16
    #change pooling to max

    classification_auxiliary_output = Average(name = 'classification_auxiliary_output')([sub_output0, sub_output1, sub_output2, sub_output3, sub_output4, sub_output5, sub_output6, sub_output7])

    classification_output = Average(name = 'classification_output')([main_classification_output,classification_auxiliary_output])

    #the same to Orientation

    # orientation_auxiliary_output = Average( name = 'orientation_auxiliary_output')([ORN_output0, ORN_output1, ORN_output2, ORN_output3, ORN_output4, ORN_output5, ORN_output6, ORN_output7])
    # orientation_output = Average(name = 'orientation_output' )([main_orientation_output,orientation_auxiliary_output])
    model = Model(inputs = l_in, outputs =classification_output)
    print('Create the Neural Net!')
    return model

In [9]:
model = get_LightNet()

Create the Neural Net!


In [10]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 8, 8, 8, 1)    0                                            
____________________________________________________________________________________________________
conv1 (Conv3D)                   (None, 7, 7, 7, 32)   288         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 7, 7, 7, 32)   128         conv1[0][0]                      
____________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)        (None, 7, 7, 7, 32)   0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [20]:
sgd = SGD(lr=0.001, momentum=cfg['momentum'], decay=0.0, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [12]:
trainModel = model.fit(patientArray,y,batch_size=32,epochs=100)

Epoch 1/100
100/100 [==============================] - 3s - loss: 2.0638 - acc: 0.2400     
Epoch 2/100
100/100 [==============================] - 1s - loss: 1.9991 - acc: 0.2400     
Epoch 3/100
100/100 [==============================] - 1s - loss: 2.0477 - acc: 0.1800     
Epoch 4/100
100/100 [==============================] - 1s - loss: 2.0027 - acc: 0.2800     
Epoch 5/100
100/100 [==============================] - 1s - loss: 1.9568 - acc: 0.2500     
Epoch 6/100
100/100 [==============================] - 1s - loss: 1.9825 - acc: 0.2300     
Epoch 7/100
100/100 [==============================] - 1s - loss: 1.9774 - acc: 0.2900     
Epoch 8/100
100/100 [==============================] - 1s - loss: 1.9957 - acc: 0.2500     
Epoch 9/100
100/100 [==============================] - 1s - loss: 1.9274 - acc: 0.3000     
Epoch 10/100
100/100 [==============================] - 1s - loss: 1.9365 - acc: 0.3000     
Epoch 11/100
100/100 [==============================] - 2s - loss: 1.9176 - acc

100/100 [==============================] - 2s - loss: 1.5834 - acc: 0.5600     
Epoch 90/100
100/100 [==============================] - 1s - loss: 1.5155 - acc: 0.6700     
Epoch 91/100
100/100 [==============================] - 2s - loss: 1.5190 - acc: 0.6900     
Epoch 92/100
100/100 [==============================] - 1s - loss: 1.5746 - acc: 0.5800     
Epoch 93/100
100/100 [==============================] - 1s - loss: 1.5704 - acc: 0.6500     
Epoch 94/100
100/100 [==============================] - 1s - loss: 1.5358 - acc: 0.6400     
Epoch 95/100
100/100 [==============================] - 1s - loss: 1.5034 - acc: 0.6300     
Epoch 96/100
100/100 [==============================] - 1s - loss: 1.5013 - acc: 0.6700     
Epoch 97/100
100/100 [==============================] - 1s - loss: 1.5363 - acc: 0.6400     
Epoch 98/100
100/100 [==============================] - 1s - loss: 1.5415 - acc: 0.6500     
Epoch 99/100
100/100 [==============================] - 1s - loss: 1.5276 - acc: 0.

In [23]:
def cross_validation(X,y):
    from sklearn import cross_validation
    k_fold = cross_validation.KFold(len(patientArray), n_folds=10)
    
    scorelist = []
    for train_indices, test_indices in k_fold:
        print('Train: %s | test: %s' % (train_indices, test_indices))
        model.fit(X[train_indices],y[train_indices],batch_size=32,epochs=1)
        score = model.evaluate(X[test_indices],y[test_indices])
        scorelist.append(score)
    result = np.mean(scorelist, axis=0)
    return result

In [24]:
result = cross_validation(patientArray,y)
print(result)

Train: [10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34
 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84
 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99] | test: [0 1 2 3 4 5 6 7 8 9]
Epoch 1/1
10/10 [==============================] - 0s
Train: [ 0  1  2  3  4  5  6  7  8  9 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34
 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84
 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99] | test: [10 11 12 13 14 15 16 17 18 19]
Epoch 1/1
10/10 [==============================] - 0s
Train: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 30 31 32 33 34
 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84
 85 86 87 88 89 90 91 9